In [13]:
"""
Direct Clustering Algorithm (DCA) - Correct Step-by-Step Implementation
Based on lexicographic sorting approach (1s move LEFT, 1s move UP)
"""

using Printf

# ============================================================================
# DISPLAY FUNCTION
# ============================================================================

function display_matrix(matrix, parts, machines, title="Machine-Part Matrix")
    """Display the matrix with row and column sums"""
    
    println("\n" * "="^75)
    println(title)
    println("="^75)
    
    n_rows, n_cols = size(matrix)
    
    # Calculate sums
    row_sums = [sum(matrix[i, :]) for i in 1:n_rows]
    col_sums = [sum(matrix[:, j]) for j in 1:n_cols]
    
    # Print header
    print("\nPart\\Mach ")
    for m in machines
        @printf("%5s", m)
    end
    println("  | # of 1s")
    println("-"^75)
    
    # Print each row
    for i in 1:n_rows
        @printf("%-10s", parts[i])
        for j in 1:n_cols
            if matrix[i, j] == 1
                print("    1")
            else
                print("    -")
            end
        end
        @printf("  |   %2d\n", row_sums[i])
    end
    
    # Print column sums
    println("-"^75)
    print("# of 1s   ")
    for cs in col_sums
        @printf("%5d", cs)
    end
    println("\n")
end

# ============================================================================
# STEP 2: INITIAL ORDERING
# ============================================================================

function step2_order_matrix(matrix, parts, machines)
    """
    Order rows in DESCENDING order by number of 1s (break ties by descending index)
    Order columns in ASCENDING order by number of 1s (break ties by descending index)
    """
    
    println("\n" * "▶"^75)
    println("STEP 2: INITIAL ORDERING")
    println("▶"^75)
    println("\nRule: Rows DESCENDING (most 1s first)")
    println("Rule: Columns ASCENDING (least 1s first)")
    println("Break ties: Descending numerical sequence")
    
    n_rows, n_cols = size(matrix)
    
    # Calculate sums
    row_sums = [sum(matrix[i, :]) for i in 1:n_rows]
    col_sums = [sum(matrix[:, j]) for j in 1:n_cols]
    
    println("\nBefore ordering:")
    println("  Parts:    ", join(parts, ", "))
    println("  Machines: ", join(machines, ", "))
    
    # Show detailed row sums
    println("\nRow sums (for ordering rows DESCENDING):")
    for i in 1:n_rows
        @printf("  %s: %d 1s\n", parts[i], row_sums[i])
    end
    
    # Show detailed column sums
    println("\nColumn sums (for ordering columns ASCENDING):")
    for j in 1:n_cols
        @printf("  %s: %d 1s\n", machines[j], col_sums[j])
    end
    
    # Sort rows: descending by sum, then descending by original index
    row_order = sortperm(1:n_rows, by = i -> (-row_sums[i], -i))
    
    # Sort columns: ascending by sum, then descending by original index
    col_order = sortperm(1:n_cols, by = j -> (col_sums[j], -j))
    
    # Apply ordering
    new_matrix = matrix[row_order, col_order]
    new_parts = parts[row_order]
    new_machines = machines[col_order]
    
    println("\n" * "-"^75)
    println("After ordering:")
    println("-"^75)
    println("  Parts:    ", join(new_parts, ", "))
    println("  Machines: ", join(new_machines, ", "))
    
    # Show row movements
    println("\nRow movements:")
    for (new_pos, old_pos) in enumerate(row_order)
        if new_pos != old_pos
            @printf("  %s (sum=%d) moved from position %d to %d\n", 
                    parts[old_pos], row_sums[old_pos], old_pos, new_pos)
        else
            @printf("  %s (sum=%d) stayed at position %d\n", 
                    parts[old_pos], row_sums[old_pos], old_pos)
        end
    end
    
    # Show column movements
    println("\nColumn movements:")
    for (new_pos, old_pos) in enumerate(col_order)
        if new_pos != old_pos
            @printf("  %s (sum=%d) moved from position %d to %d\n", 
                    machines[old_pos], col_sums[old_pos], old_pos, new_pos)
        else
            @printf("  %s (sum=%d) stayed at position %d\n", 
                    machines[old_pos], col_sums[old_pos], old_pos)
        end
    end
    
    display_matrix(new_matrix, new_parts, new_machines, 
                   "After STEP 2: Ordered Matrix")
    
    return new_matrix, new_parts, new_machines
end

# ============================================================================
# STEP 3: SORT COLUMNS LEXICOGRAPHICALLY
# ============================================================================

function step3_sort_columns(matrix, parts, machines)
    """
    Sort columns lexicographically by all rows (DESCENDING: 1s come before 0s)
    For each column, create tuple: (row1, row2, ..., rowM)
    Sort columns so that columns with more 1s (especially in early rows) come LEFT
    """
    
    println("\n" * "▶"^75)
    println("STEP 3: COLUMN SORTING (Lexicographic - 1s move LEFT)")
    println("▶"^75)
    
    n_rows, n_cols = size(matrix)
    
    # Create tuple for each column
    column_tuples = [Tuple(matrix[:, j]) for j in 1:n_cols]
    
    println("\nColumn tuples (current order):")
    for j in 1:n_cols
        println("  $(machines[j]): $(column_tuples[j])")
    end
    
    # Sort columns by tuples in DESCENDING order
    # This means columns with 1 in row 1 come before columns with 0 in row 1
    # If tied, compare row 2, etc.
    col_order = sortperm(column_tuples, rev=true)
    
    # Check if changed
    updated = (col_order != 1:n_cols)
    
    if updated
        new_matrix = matrix[:, col_order]
        new_machines = machines[col_order]
        
        println("\n✓ Columns were reordered!")
        println("Old machine order: ", join(machines, ", "))
        println("New machine order: ", join(new_machines, ", "))
        
        # Show which columns moved
        println("\nColumn movements:")
        for (new_pos, old_pos) in enumerate(col_order)
            if new_pos != old_pos
                println("  $(machines[old_pos]) moved from position $old_pos to $new_pos")
            end
        end
        
        display_matrix(new_matrix, parts, new_machines, 
                      "After STEP 3: Column Sorted")
        
        return new_matrix, parts, new_machines, true
    else
        println("\n✓ No column reordering needed (already optimal)")
        return matrix, parts, machines, false
    end
end

# ============================================================================
# STEP 4: SORT ROWS LEXICOGRAPHICALLY
# ============================================================================

function step4_sort_rows(matrix, parts, machines)
    """
    Sort rows lexicographically by all columns (DESCENDING: 1s come before 0s)
    For each row, create tuple: (col1, col2, ..., colN)
    Sort rows so that rows with more 1s (especially in early columns) come UP
    """
    
    println("\n" * "▶"^75)
    println("STEP 4: ROW SORTING (Lexicographic - 1s move UP)")
    println("▶"^75)
    
    n_rows, n_cols = size(matrix)
    
    # Create tuple for each row
    row_tuples = [Tuple(matrix[i, :]) for i in 1:n_rows]
    
    println("\nRow tuples (current order):")
    for i in 1:n_rows
        println("  $(parts[i]): $(row_tuples[i])")
    end
    
    # Sort rows by tuples in DESCENDING order
    # This means rows with 1 in col 1 come before rows with 0 in col 1
    # If tied, compare col 2, etc.
    row_order = sortperm(row_tuples, rev=true)
    
    # Check if changed
    updated = (row_order != 1:n_rows)
    
    if updated
        new_matrix = matrix[row_order, :]
        new_parts = parts[row_order]
        
        println("\n✓ Rows were reordered!")
        println("Old part order: ", join(parts, ", "))
        println("New part order: ", join(new_parts, ", "))
        
        # Show which rows moved
        println("\nRow movements:")
        for (new_pos, old_pos) in enumerate(row_order)
            if new_pos != old_pos
                println("  $(parts[old_pos]) moved from position $old_pos to $new_pos")
            end
        end
        
        display_matrix(new_matrix, new_parts, machines, 
                      "After STEP 4: Row Sorted")
        
        return new_matrix, new_parts, machines, true
    else
        println("\n✓ No row reordering needed (already optimal)")
        return matrix, parts, machines, false
    end
end

# ============================================================================
# STEP 5: ITERATE UNTIL CONVERGENCE
# ============================================================================

function step5_iterate_until_convergence(matrix, parts, machines)
    """
    Repeat Steps 3-4 until no more changes occur
    """
    
    println("\n" * "▶"^75)
    println("STEP 5: ITERATE UNTIL CONVERGENCE")
    println("▶"^75)
    
    current_matrix = matrix
    current_parts = parts
    current_machines = machines
    iteration = 1
    max_iterations = 20
    
    while iteration <= max_iterations
        println("\n" * "+"^75)
        println("ITERATION #$iteration")
        println("+"^75)
        
        # Step 3: Sort columns
        new_matrix, new_parts, new_machines, col_updated = 
            step3_sort_columns(current_matrix, current_parts, current_machines)
        
        # Step 4: Sort rows
        new_matrix, new_parts, new_machines, row_updated = 
            step4_sort_rows(new_matrix, new_parts, new_machines)
        
        # Check convergence
        if !col_updated && !row_updated
            println("\n" * "✓"^75)
            println("✓ CONVERGENCE ACHIEVED! No more changes possible.")
            println("✓"^75)
            break
        end
        
        current_matrix = new_matrix
        current_parts = new_parts
        current_machines = new_machines
        iteration += 1
    end
    
    if iteration > max_iterations
        println("\n⚠ Warning: Maximum iterations reached!")
    end
    
    return current_matrix, current_parts, current_machines
end

# ============================================================================
# STEP 6: IDENTIFY MANUFACTURING CELLS
# ============================================================================

function step6_identify_cells(matrix, parts, machines)
    """
    Identify block diagonal structure and form manufacturing cells
    Using cluster identification algorithm (cross method)
    """
    
    println("\n" * "▶"^75)
    println("STEP 6: IDENTIFY MANUFACTURING CELLS")
    println("▶"^75)
    
    n_rows, n_cols = size(matrix)
    visited_rows = falses(n_rows)
    visited_cols = falses(n_cols)
    cells = []
    
    # Cluster identification algorithm
    for start_row in 1:n_rows
        if visited_rows[start_row]
            continue
        end
        
        cell_parts = Int[]
        cell_machines = Int[]
        
        # BFS-like expansion
        to_check_parts = [start_row]
        
        while !isempty(to_check_parts)
            current_part = popfirst!(to_check_parts)
            
            if current_part in cell_parts
                continue
            end
            
            push!(cell_parts, current_part)
            visited_rows[current_part] = true
            
            # Find machines used by this part
            for col in 1:n_cols
                if matrix[current_part, col] == 1 && !(col in cell_machines)
                    push!(cell_machines, col)
                    visited_cols[col] = true
                    
                    # Find other parts using this machine
                    for row in 1:n_rows
                        if matrix[row, col] == 1 && !visited_rows[row]
                            push!(to_check_parts, row)
                        end
                    end
                end
            end
        end
        
        sort!(cell_parts)
        sort!(cell_machines)
        push!(cells, (parts=cell_parts, machines=cell_machines))
    end
    
    # Display results
    println("\n" * "🔷"^75)
    println("CELL FORMATION RESULTS")
    println("🔷"^75)
    println("\nNumber of cells formed: $(length(cells))")
    
    for (i, cell) in enumerate(cells)
        println("\n" * "-"^75)
        println("CELL #$i:")
        println("-"^75)
        println("  Machines: ", join([machines[m] for m in cell.machines], ", "))
        println("  Parts:    ", join([parts[p] for p in cell.parts], ", "))
        println("  Size:     $(length(cell.parts)) parts × $(length(cell.machines)) machines")
    end
    
    # Display final matrix with cell markings
    display_final_matrix_with_cells(matrix, parts, machines, cells)
    
    return cells
end

function display_final_matrix_with_cells(matrix, parts, machines, cells)
    """Display the final matrix with cell numbers marked"""
    
    println("\n" * "="^75)
    println("FINAL MATRIX WITH CELL BOUNDARIES")
    println("="^75)
    
    n_rows, n_cols = size(matrix)
    
    # Create cell assignment map
    cell_map = zeros(Int, n_rows, n_cols)
    for (cell_num, cell) in enumerate(cells)
        for r in cell.parts, c in cell.machines
            if matrix[r, c] == 1
                cell_map[r, c] = cell_num
            end
        end
    end
    
    # Print header
    print("\nPart\\Mach ")
    for m in machines
        @printf("%6s", m)
    end
    println()
    println("-"^75)
    
    # Print rows
    for i in 1:n_rows
        @printf("%-10s", parts[i])
        for j in 1:n_cols
            if matrix[i, j] == 1
                @printf("  [%2d]", cell_map[i, j])
            else
                print("     -")
            end
        end
        println()
    end
    
    println("\nNote: [1], [2], etc. indicate cell membership")
end

# ============================================================================
# MAIN DCA FUNCTION
# ============================================================================

function run_dca(matrix, parts, machines)
    """
    Main function to run the complete DCA algorithm
    """
    
    println("\n" * "█"^75)
    println("█" * " "^15 * "DIRECT CLUSTERING ALGORITHM (DCA)" * " "^25 * "█")
    println("█"^75)
    
    # Step 1: Display initial matrix
    display_matrix(matrix, parts, machines, "STEP 1: Initial Machine-Part Matrix")
    
    # Step 2: Initial ordering
    matrix, parts, machines = step2_order_matrix(matrix, parts, machines)
    
    # Steps 3-5: Iterative sorting
    matrix, parts, machines = step5_iterate_until_convergence(matrix, parts, machines)
    
    # Step 6: Identify cells
    cells = step6_identify_cells(matrix, parts, machines)
    
    println("\n" * "█"^75)
    println("█" * " "^30 * "DCA COMPLETED!" * " "^30 * "█")
    println("█"^75)
    
    return matrix, parts, machines, cells
end

run_dca (generic function with 1 method)

In [14]:
# ============================================================================
# (10 Products × 7 Machines)
# ============================================================================

println("\n\n" * "🔷"^37)
println("🔷 10 Products × 7 Machines")
println("🔷"^37)

# Data from your corrected table
matrix3 = [
    0 1 0 1 0 0 1;   # A100-001
    0 0 0 1 0 0 1;   # A200-110
    0 0 0 0 1 1 1;   # A300-021
    0 0 0 0 1 1 0;   # B150-100
    1 0 1 0 0 0 1;   # B160-105
    1 0 1 0 0 0 0;   # B275-110
    1 0 1 0 0 0 0;   # C120-001
    0 1 0 0 1 0 1;   # C150-100
    0 0 1 0 1 0 0;   # C203-105
    1 1 0 0 0 0 0    # C205-106
]

parts3 = [
    "A100-001",
    "A200-110",
    "A300-021",
    "B150-100",
    "B160-105",
    "B275-110",
    "C120-001",
    "C150-100",
    "C203-105",
    "C205-106"
]

machines3 = ["Drill", "Drill2", "Lathe", "Lathe2", "Mill", "Oven", "Wash"]

final_matrix3, final_parts3, final_machines3, cells3 =
    run_dca(matrix3, parts3, machines3)

println("\n\nVERIFICATION:")
println("Final machine order: ", join(final_machines3, ", "))
println("Final part order:    ", join(final_parts3, ", "))

println("\n\n" * "✅"^37)
println("✅ COMPLETED!")
println("✅"^37)




🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷
🔷 10 Products × 7 Machines
🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷

███████████████████████████████████████████████████████████████████████████
█               DIRECT CLUSTERING ALGORITHM (DCA)                         █
███████████████████████████████████████████████████████████████████████████

STEP 1: Initial Machine-Part Matrix

Part\Mach DrillDrill2LatheLathe2 Mill Oven Wash  | # of 1s
---------------------------------------------------------------------------
A100-001      -    1    -    1    -    -    1  |    3
A200-110      -    -    -    1    -    -    1  |    2
A300-021      -    -    -    -    1    1    1  |    3
B150-100      -    -    -    -    1    1    -  |    2
B160-105      1    -    1    -    -    -    1  |    3
B275-110      1    -    1    -    -    -    -  |    2
C120-001      1    -    1    -    -    -    -  |    2
C150-100      -    1    -    -    1    -    1  |    3
C203-105      -    -    1    -    1    -    -  |    2
C205-10

In [8]:
# ============================================================================
# (10 Products × 12 Machines)
# ============================================================================

println("\n\n" * "🔷"^37)
println("🔷 EXAMPLE 3: Custom Data - 10 Products × 12 Machines")
println("🔷"^37)

# Corrected 10 × 12 binary matrix
matrix3 = [
    0 0 1 1 0 1 0 0 0 0 0 0;  # C150
    0 1 0 1 0 0 1 0 0 0 0 0;  # A300
    1 0 0 0 1 0 0 0 1 0 0 0;  # A100
    1 0 0 0 0 0 0 0 1 0 0 0;  # A200
    0 0 1 0 0 0 0 0 0 1 0 1;  # B160
    0 0 0 1 0 0 0 1 0 0 0 0;  # B150
    0 0 0 1 0 0 0 0 0 0 1 0;  # C203
    0 0 0 0 0 1 0 0 0 0 0 1;  # C205
    0 0 0 0 0 0 0 0 0 1 0 1;  # C120
    0 0 0 0 0 0 0 0 0 1 0 1   # B275
]

# Part names
parts3 = [
    "C150", "A300", "A100", "A200", "B160",
    "B150", "C203", "C205", "C120", "B275"
]

# 12 machines
machines3 = [
    "Wash_A","Wash_B","Wash_C","Mill","Drill2_A","Drill2_B",
    "Oven_A","Oven_B","Lathe2","Lathe_A","Lathe_B","Drill"
]

# Run DCA
final_matrix3, final_parts3, final_machines3, cells3 =
    run_dca(matrix3, parts3, machines3)

println("\n\nVERIFICATION:")
println("Final machine order: ", join(final_machines3, ", "))
println("Final part order:    ", join(final_parts3, ", "))

println("\n\n" * "✅"^37)
println("✅ COMPLETED!")
println("✅"^37)

#now fill it inside that code both machinbe and part



🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷
🔷 EXAMPLE 3: Custom Data - 10 Products × 12 Machines
🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷

███████████████████████████████████████████████████████████████████████████
█               DIRECT CLUSTERING ALGORITHM (DCA)                         █
███████████████████████████████████████████████████████████████████████████

STEP 1: Initial Machine-Part Matrix

Part\Mach Wash_AWash_BWash_C MillDrill2_ADrill2_BOven_AOven_BLathe2Lathe_ALathe_BDrill  | # of 1s
---------------------------------------------------------------------------
C150          -    -    1    1    -    1    -    -    -    -    -    -  |    3
A300          -    1    -    1    -    -    1    -    -    -    -    -  |    3
A100          1    -    -    -    1    -    -    -    1    -    -    -  |    3
A200          1    -    -    -    -    -    -    -    1    -    -    -  |    2
B160          -    -    1    -    -    -    -    -    -    1    -    1  |    3
B150          -    -    -    1   

In [10]:
# ============================================================================
# (10 Products × 12 Machines)
# ============================================================================

println("\n\n" * "🔷"^37)
println("🔷 10 Products × 12 Machines")
println("🔷"^37)

# Corrected 10 × 12 binary matrix
matrix3 = [
    1 0 1 0 1 0 0 0 0 0 0 0 ;  # C150
    1 0 1 0 0 0 0 1 0 0 0 0 ;  # A300
    0 1 0 0 0 1 0 0 1 0 0 0 ;  # A100
    0 1 0 0 0 0 0 0 1 0 0 0 ;  # A200
    1 0 0 0 0 0 0 0 0 0 1 1 ;  # B160
    0 0 0 1 0 0 1 0 0 0 0 0 ;  # B150
    0 0 0 1 0 0 0 0 0 1 0 0 ;  # C203
    0 0 0 0 1 0 0 0 0 0 0 1 ;  # C205
    0 0 0 0 0 0 0 0 0 0 1 1 ;  # C120
    0 0 0 0 0 0 0 0 0 0 1 1    # B275
]

# Part names
parts3 = [
    "C150", "A300", "A100", "A200", "B160",
    "B150", "C203", "C205", "C120", "B275"
]

# 12 machines
machines3 = ["Wash_A", "Wash_B",
 "Mill_A", "Mill_B",
 "Drill2_A", "Drill2_B",
 "Oven_A", "Oven_B",
 "Lathe2", "Lathe_A", "Lathe_B",
 "Drill"]

# Run DCA
final_matrix3, final_parts3, final_machines3, cells3 =
    run_dca(matrix3, parts3, machines3)

println("\n\nVERIFICATION:")
println("Final machine order: ", join(final_machines3, ", "))
println("Final part order:    ", join(final_parts3, ", "))

println("\n\n" * "✅"^37)
println("✅ COMPLETED!")
println("✅"^37)




🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷
🔷 10 Products × 12 Machines
🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷

███████████████████████████████████████████████████████████████████████████
█               DIRECT CLUSTERING ALGORITHM (DCA)                         █
███████████████████████████████████████████████████████████████████████████

STEP 1: Initial Machine-Part Matrix

Part\Mach Wash_AWash_BMill_AMill_BDrill2_ADrill2_BOven_AOven_BLathe2Lathe_ALathe_BDrill  | # of 1s
---------------------------------------------------------------------------
C150          1    -    1    -    1    -    -    -    -    -    -    -  |    3
A300          1    -    1    -    -    -    -    1    -    -    -    -  |    3
A100          -    1    -    -    -    1    -    -    1    -    -    -  |    3
A200          -    1    -    -    -    -    -    -    1    -    -    -  |    2
B160          1    -    -    -    -    -    -    -    -    -    1    1  |    3
B150          -    -    -    1    -    -    1    -    -  